# Using TorchServe with Containers with CPU-based Image
This introductory lab will take you through the following hands-on exercises:

* [Install TorchServe](https://github.com/pytorch/serve#install-torchserve) and it's dependencies on an Amazon SageMaker Notebook
* Create a TorchServe Docker image (sans model)
* Create TorchServe Docker image from source
* Create torch-model-archiver from container
* Start TorchServe using a Docker container
* Performing Inference using a TorchServe container
* Stop the TorchServe Container
* Hints for running TorchServe in a Production Docker Environment 
* Cleanup

## Preparing the TorchServe environment on SageMaker
Install the prerequisites. Most of these are the same setup steps that were conducted in Lab 01. 

In [ ]:
!chmod +x ./scripts/prerequisites.sh
!./scripts/prerequisites.sh

## Create TorchServe Docker CPU image
Create a docker image.

In [ ]:
%%bash
cd serve/docker
DOCKER_BUILDKIT=1 docker build --file Dockerfile -t torchserve:latest .

Start the container with a TorchServe image on ports 8080 and 8081 exposed.

In [ ]:
%%bash
cd serve/docker
docker run -d --rm -it \
    --name no_model_container \
    -p 8080:8080 \
    -p 8081:8081 \
    pytorch/torchserve:latest

TorchServe inference and management APIs are accessible on localhost ports 8080 and 8081. The status from the following endpoint command should should read "Healthy".

In [ ]:
!curl http://localhost:8080/ping

Note that that this container does not contain a model which one can see via the following command.

In [ ]:
!curl http://localhost:8081/models

We will now stop this container in order to demonstrate starting a container including a model.

In [ ]:
# Stop the TorchServe container.
!docker stop no_model_container

## Create TorchServe Docker image from source
The following are examples on how to use the build_image.sh script to build Docker images from source to support CPU or GPU inference.

To create a Docker image for a specific branch, use the following command:

   `./build_image.sh -b <branch_name>`

In [ ]:
%%bash
cd serve/docker
./build_image.sh -b master

The provided [docker file](serve/docker/Dockerfile) can be used to build images for both CPU and GPU environments.

## Create torch-model-archiver from container
We start by copying a sample script to the docker subdirectory. This script file can be used to start both CPU and GPU based images. You can download this [script file for viewing](../source/start_densenet.sh).

In [ ]:
!cp ./scripts/start_densenet.sh ./serve/docker/.
!chmod +x serve/docker/start_densenet.sh

## Start TorchServe using a Docker container
To start TorchServe inside the container with a pre-registered densenet-161 image classification model, use the following commands:

In [ ]:
!serve/docker/start_densenet.sh

You may then query the list of registered models to verify our pre=trained densenet_161 model is also being served.

In [ ]:
!curl http://localhost:8081/models

## Performing Inference using a TorchServe container
To test the TorchServe model server, you just need to send a request to the Inference API. Let's start by pulling down an image of a [Proboscis Monkey](https://en.wikipedia.org/wiki/Proboscis_monkey) and a [Tiger Beetle](https://en.wikipedia.org/wiki/Tiger_beetle).
<img src="https://torchserve-workshop.s3.amazonaws.com/proboscis-monkey-tiger-beetle-grouped.png">


In [ ]:
!curl -O https://torchserve-workshop.s3.amazonaws.com/proboscis-monkey.jpg
!curl -O https://torchserve-workshop.s3.amazonaws.com/tiger-beetle.jpg

Now that we have two images, we can use curl to send POST to the TorchServe predict endpoint with our images. The predictions endpoint returns a prediction response in JSON. With both the Proboscis Money and the Tiger Beetle, we see several different prediction types along with their associated confidence scores of each prediction.

In [ ]:
!curl -X POST http://localhost:8080/predictions/densenet161 -T proboscis-monkey.jpg
!curl -X POST http://localhost:8080/predictions/densenet161 -T tiger-beetle.jpg

You will see that these results are the same as seen with Lab 01 but here were are using a container rather than SageMaker local mode.

## Stop the TorchServe Container
Now, shut down the TorchServe container using this command: `docker container stop <container_name | contain id>`.

In [ ]:
!docker container stop torchserve_container

## Hints for running TorchServe in a Production Docker Environment
You may wish to consider these options when deploying TorchServe with Docker in a production environment.
- Shared Memory Size: The `shm-size` parameter allows you to specify the shared memory that a container can use. It enables memory-intensive containers to run faster by giving more access to allocated memory.
  

- User Limits for System Resources
   - `--ulimit memlock=-1`: Maximum locked-in-memory address space.
   - `--ulimit stack`: Linux stack size
   
    The current ulimit values can be viewed by executing ulimit -a. A more exhaustive set of options for resource constraining can be found in the Docker Documentation:
    - [Resource constraints](https://docs.docker.com/config/containers/resource_constraints/),
    - [Set ulimits in a container](https://docs.docker.com/engine/reference/commandline/run/#set-ulimits-in-container---ulimit), and
    - [Runtime constraints](https://docs.docker.com/engine/reference/run/#runtime-constraints-on-resources)
    

- Exposing specific ports / volumes between the host & docker env.

    - `-p8080:p8080 -p8081:8081` TorchServe uses default ports 8080 / 8081 for inference & management APIs. You may want to expose these ports to the host for HTTP Requests between Docker & Host.
    - The model store is passed to torchserve with the --model-store option. You may want to consider using a shared volume if you prefer pre populating models in model-store directory.
    
    
Here is an example using all of these options for production use,

```
docker run -d --rm \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -p8080:8080 \
    -p8081:8081 \
    --mount type=bind,source=/path/to/model/store,target=/tmp/models <container> torchserve 
    --model-store=/tmp/models 
```

## Cleanup
The next step removes files created during this lab.

In [ ]:
!chmod +x ./scripts/cleanup.sh
!./scripts/cleanup.sh